## Read earnings reports (OO version)

E. Quinn 12/22/2019

This notebook uses pdfminer to extract the information from the individual earnings report

The documentation for pdfminer is at:

https://buildmedia.readthedocs.org/media/pdf/pdfminer-docs/latest/pdfminer-docs.pdf

Maintenance:

* 3/6/2020  
  * Add check date and number
* 3/7/2020  
  * Align personnel classes with support professionals structure
  * Implement salary step capture for support professionals
* 4/8/2020
  * Rewrite logic to base data structure on check number and check date
  * Simplify payment decoding logic to take advantage of having check date
  * Data corrections for check dates and numbers:
    * Adjust 5 check dates to aliign with nearest payday
    * Generate 4 artificial check numbers for zero earnings lines
* 4/18/2020
  * Add FY2016, FY2015 and FY2019+FY2020ytd (!)
  * Add code to move 12/25/2015 payroll to 12/24/2015 
    
To do:
* Replace computed salary matrix for FY2020 with numbers from contract (correct small rounding errors)

## Import standard python datascience packages

In [1]:
import sys
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cloudpickle
%matplotlib inline

In [2]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

In [3]:
sys.path.append("/home/gquinn/EG/school_committee/py_EGSC")

In [4]:
import SC_classes as pySC

### Show the directory we are running in

In [5]:
!pwd

/home/gquinn/EG/school_committee/finance_subcommittee/notebooks


### Load RIDE UCOA labels 

In [6]:
UCOA_labels = pySC.UCOA_labels()
    
help(UCOA_labels)

Help on UCOA_labels in module SC_classes object:

class UCOA_labels(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      UCOA_labels class provides labels for UCOA fields
 |  
 |  get_label(self, col, code)
 |      Usage: get_label(col,code) returns the code for field col
 |  
 |  get_labels_dictionary(self)
 |      Returns the UCOA labels dictionary
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



### EG accounting codes class

provides descriptions for EG accounting codes and mapping to UCOA codes

In [7]:
EG_acct_codes = pySC.EG_acct_codes()
    
help(EG_acct_codes)

Help on EG_acct_codes in module SC_classes object:

class EG_acct_codes(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  check_eg_acct_desc6(self, acct)
 |      check whether first 6 digits map an acct6 key
 |  
 |  get_UCOA_from_acct(self, acct, ucl)
 |  
 |  get_UCOA_from_acct6(self, acct6, ucl)
 |  
 |  get_eg_acct_UCOA(self, acct)
 |      Provides UCOA codes for accounting codes in EG MUNIS system.
 |  
 |  get_eg_acct_codes(self)
 |      Returns dictionary of account codes.
 |  
 |  get_eg_acct_desc(self, acct)
 |      Provides descriptions for accounting codes in EG MUNIS system.
 |  
 |  get_eg_acct_desc6(self, acct)
 |      Provides descriptions for accounting codes in EG MUNIS system.
 |  
 |  get_loc_from_acct(self, acct, ucl)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for

### Teacher salary matrix

In [8]:
teacher_salary_matrix = pySC.teacher_salary_matrix()
    
help(teacher_salary_matrix)

Help on teacher_salary_matrix in module SC_classes object:

class teacher_salary_matrix(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  decode_earnings(self, check_date, rate, in_earnings, obj)
 |  
 |  get_cba_matrix(self)
 |  
 |  get_future_code_from_code(self, code, incr)
 |      Returns CBA salary given step code for FY2013-FY2022.
 |  
 |  get_future_salary_from_code(self, code, incr)
 |      Returns CBA salary given step code for FY2013-FY2022.
 |  
 |  get_salary(self, fyear, step, col)
 |      Returns CBA salary given fiscal year, column, and step for FY2013-FY2022.
 |  
 |  get_salary_from_code(self, code)
 |      Returns CBA salary given step code for FY2013-FY2022.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |    

## Import pdfminer packages

In [9]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

## Read the pdf and create a dictionary with the contents of each text box

### Function read_pdf() reads a PDF and returns a dictionary containing the contents

Strategy for this document:  

Save information from each element in the LTTextBox objects in a dictionary including:

- x0 horizontal coordinate of the upper left corner of the text box
- x1 horizontal coordinate of the lower right corner of the text box
- y0 vertical coordinate of the upper left corner of the text box
- y1 vertical coordinate of the lower right corner of the text box
- page number 
- sequence number of text box within this page
- text contained in the text box, converted to ascii

Parsing the text is complicated by the fact that that a text box may span multiple columns and/or rows, and the text box groupings vary quite a bit depending on the page contents and layout.

However, with a bit of luck the structure of the document will allow the contents to be deciphered with the following heuristics:

- Text boxes containing left justified columns will tend to have nearly the same x0 coordinates
- Text boxes containing right justified columns will tend to have nearly the same x1 coordinates
- The codes for fund, account code, and object code are numeric and have fixed lengths
- Extraneous information is often preceded or followed by a series of underscore and newline characters
- Last name can be distinguished because is the only field that is all characters followed by a comma
- Last name may be preceded by between one and three numerical fields:  fund, account, object.  If it is, the x0 value is shifted to the left.
    - Three numerical fields precede the name:  assume they are fund, account, object
    - Two numerical fields precede the name: assume they are account, object
    - One numerical field precedes the name: assume it is object
    

In [10]:
def read_pdf(path):
    document = open(path, 'rb')                                     #read a pdf and create a document object
    rsrcmgr = PDFResourceManager()                                  #create a resource manager
    laparams = LAParams()                                           #set the parameters for analysis
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)          #create a PDF page aggregator object
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    pdf={}                                                          #dictionary to hold the results

    pageno = -1                                                     #initialize page coounter to zero

    for page in PDFPage.get_pages(document):                        #loop through the pdf page by page
        pageno = pageno + 1                                         #increment the page number
        pdf[pageno] = {}                                            #dictionary for this page
        interpreter.process_page(page)                              # receive the LTPage object for the page.
        layout = device.get_result()                                # create layout object
        tbox_no=0                                                   # index for element number
        for element in layout:
            if (type(element).__name__=='LTTextBoxHorizontal'):     #loop through text boxes
                tbox_no += 1                                        #increment text box number
                pdf[pageno][tbox_no] = {}                           #dictionary for text boxes within page
                x0 = round(element.x0,2)                            #x0 coordinate of textbox corner
                x1 = round(element.x1,2)                            #x1 coordinate of textbox corner
                y0 = round(element.y0,2)                            #y0 coordinate of textbox corner
                y1 = round(element.y1,2)                            #y1 coordinate of textbox corner
                txt = element.get_text().encode('ascii', 'ignore')  #text converted to ascii
                pdf[pageno][tbox_no]['x0'] = x0                     #create x0 coordinate entry
                pdf[pageno][tbox_no]['x1'] = x1                     #create x1 coordinate entry
                pdf[pageno][tbox_no]['y0'] = y0                     #create y0 coordinate entry
                pdf[pageno][tbox_no]['y1'] = y1                     #create y1 coordinate entry

                pdf[pageno][tbox_no]['text'] = ''.join(chr(c) for c in txt) #convert bytes to string
    return(pdf)

### Utility functions

In [11]:
#remove the commas from earnings and rate values

def remove_commas(st):
    newstr = st.replace(',','')                     #remove commas from string
    return(newstr)

In [12]:
#remove the headings fields 

def remove_headings(st):
    lines = st.split('\n')                         #split the string at newline characters '\n'
    for line in lines:                             #loop through the resulting lines
        if (line.startswith('FUND ') |\
           (line.startswith('POSITION')) |\
           (line.startswith('RATE')) |\
           (line.startswith('ACCT-')) |\
           (line.startswith('CHECK')) |\
           (line.startswith('_'))):                #check for strings that appear only in headings
            try:
                newline_index = st.index('\n')     #if present, remove this line from the text string
                st = st[newline_index+1:]
            except ValueError:
                print('Value Error',st)            #recover from Value Error and print string
        else:
            return(st)                             #if no headings, just return
    return('')

### Read the FY2015 earnings report

In [13]:
#p15 = read_pdf('../earnings/Munis_7-1-2014_to_6-30-2015.pdf')

### Read the FY2016 earnings report

In [14]:
#p16 = read_pdf('../earnings/Munis_7-1-2015_to_6-30-2016.pdf')

### Read the FY2017 earnings report

In [15]:
#p17 = read_pdf('../FY17 Gene_Redacted.pdf')
#p17 = read_pdf('../earnings/Munis_7-1-2016_to_6-30-2017.pdf')

### Read the FY2018 earnings report

In [16]:
#p18 = read_pdf('../FY18 Gene_Redacted.pdf')
#p18 = read_pdf('../earnings/Munis_7-1-2017_to_6-30-2018.pdf')

### Read the FY2019 earnings report

In [17]:
#p19 = read_pdf('../earnings/Munis_7-1-2018_to_7-1-2019.pdf')

### Read the FY2020 through current report

In [18]:
p20 = read_pdf('../earnings/Munis_7-1-2019_to_current.pdf')

### Build a dictionary with only those text boxes containing names

Use the following algorithm to identify text boxes that contain names:

- x0, horizontal coordinate of the upper left corner of the text box, is less than 162
- the text string contains at least one comma

In [19]:
def get_names(dct):

    dnames = {}

    fund = ''
    acct = ''
    obj  = ''
    
    for page in sorted(dct.keys()):                                #loop through text box dictionary by page # 
        if (page not in dnames.keys()):                            #page number is highest level key
            dnames[page] = {}                                      #initialize entry for this page
        for tb in sorted(dct[page].keys()):                        #loop through all text boxes on this page
            if (dct[page][tb]['x0'] < 162.0):                      #those with names start to the left of x0=162
                txt = str(dct[page][tb]['text'])                   #convert the 'text' element to a string
                if (',' in txt):                                   #every name contains a comma
                    txt = remove_headings(txt)
                    lines = txt.split('\n')                        #split text into lines
                    words = lines[0].split()                       #split first line into words
                    for word in words:                             #loop through and strip out fund, acct, obj
                        if (word.isdigit()):
                            if (len(word)==4):                     # 4 digits means fund
                                fund = word
                            if (len(word)==8):                     # 8 digits means acct-code
                                acct = word
                            if (len(word)==5):                     # 5 digits means obj
                                obj = word
                            txt = txt[len(word)+1:]                # remove fund/acct/obj from txt
                    dnames[page][tb] = {}                          #initialize dictionary for this page
                    dnames[page][tb]['x0'] = dct[page][tb]['x0']
                    dnames[page][tb]['x1'] = dct[page][tb]['x1']
                    dnames[page][tb]['y0'] = dct[page][tb]['y0']
                    dnames[page][tb]['y1'] = dct[page][tb]['y1']
                    dnames[page][tb]['fund'] = fund
                    dnames[page][tb]['acct'] = acct
                    dnames[page][tb]['obj'] = obj
                    dnames[page][tb]['text'] = txt
    return(dnames)

### Consolidate text boxes that overlap on the vertical scale and contain names

In [20]:
def consolidate_name_boxes(names):
    newnames = {}
    
    for page in sorted(names.keys()):                                        #loop through pages of pdf
        newnames[page] = {}                                                  #initialize new names dictionary
        skip = make_array()                                                  #initialize list of boxes to skip
    
        for tb in sorted(names[page].keys()):                                #loop through text boxes on this page
            for tb2 in sorted(names[page].keys()):                           #compare this one to the others
                if ((tb2 > tb) & \
                    (names[page][tb]['y0'] <= names[page][tb2]['y1']) & \
                    (names[page][tb2]['y0'] <= names[page][tb]['y1'])):      
                    d = {}                                                   #initialize replacement entry
                    d['x0'] = names[page][tb]['x0']                          #keep x0    
                    d['x1'] = names[page][tb2]['x1']                         #replace x1 with tb2 value
                    d['y0'] = names[page][tb2]['y0']                         #replace y0 with tb2 value
                    d['y1'] = names[page][tb]['y1']                          #keep y1 value
                    d['text'] = names[page][tb]['text'] +\
                        names[page][tb2]['text']                             #contatenate text strings
                    d['fund'] = names[page][tb]['fund']                      #copy fund, acct, and obj
                    d['acct'] = names[page][tb]['acct']
                    d['obj'] = names[page][tb]['obj']
                    newnames[page][tb2] = d                                  #plug into dictionary
                    skip = np.append(skip,tb)                                #add old boxes to skip list
                    skip = np.append(skip,tb2)
            if (tb not in skip):                                             #if no match, check skip list 
                newnames[page][tb] = names[page][tb]                         #just copy if not in skip list
                    
    return(newnames)

In [21]:
def combdd(cn,pdf):
    
    dd = {}
    
    for page in sorted(cn.keys()):
        if page not in dd.keys():
            dd[page] = {}
        for tb in sorted(cn[page].keys()):                               #loop through consolidated name textboxes
            dd[page][tb] = cn[page][tb]
            y0  = dd[page][tb]['y0']                                      #extract vertical coordinates
            y1  = dd[page][tb]['y1']
            txt = dd[page][tb]['text']                           #extract text
            for tb2 in sorted(pdf[page].keys()):                            #loop through the other boxes in pdf
                if (tb != tb2):                                             #ignore if same box as names
                    tx0 = pdf[page][tb2]['x0']                              #get horizontal offset
                    ty0 = pdf[page][tb2]['y0']                              #check whether the vertical 
                    ty1 = pdf[page][tb2]['y1']                              #range of this box overlaps that
                    if ((y0 <= ty1) & (ty0 <= y1)):                         #of the name box
                        txt = remove_headings(pdf[page][tb2]['text'])
                        if ((312.0 < tx0) & (tx0 < 316.0)):                 #match to DATE/NUMBER
                            dd[page][tb]['numbers1'] = txt
                        if ((383.0 < tx0) & (tx0 < 395.0)):                 #match to NUMBER
                            if 'numbers2' not in dd[page][tb].keys():
                                dd[page][tb]['numbers2'] = txt
                            else:
                                dd[page][tb]['numbers2'] += txt
                        if ((437.0 < tx0) & (tx0 < 440.0)):                 #match to POSITION
                            dd[page][tb]['positions'] = txt
                        if ((509.0 < tx0) & (tx0 < 533.0)):                 #match to RATE 
                            dd[page][tb]['rates'] = remove_commas(txt)
                        if ((558.0 < tx0) & (tx0 < 630.0)):                 #match to ACCT-EARNINGS
                            dd[page][tb]['earnings'] = remove_commas(txt)

    return(dd)

In [22]:
def get_lines(nn):
    
    lld = {}
    
    for page in sorted(nn.keys()):
        if page not in lld.keys():
            lld[page] = {}
        for tb in sorted(nn[page].keys()):
            if tb not in lld[page].keys():
                lld[page][tb]              = {}
                lld[page][tb]['names']     = []
                lld[page][tb]['checks']    = []
                lld[page][tb]['dates']     = []
                lld[page][tb]['rates']     = []
                lld[page][tb]['earnings']  = []
                lld[page][tb]['positions'] = []
                lld[page][tb]['fund']      = ''
                lld[page][tb]['acct']      = ''
                lld[page][tb]['obj']       = ''
            txt = nn[page][tb]['text']
            words = txt.split('\n')
            for word in words:
                if (len(word) > 1):
                    lld[page][tb]['names'].append(word)
            if 'numbers1' in nn[page][tb].keys():
                txt = nn[page][tb]['numbers1']
                words = txt.split('\n')
                for word in words:
                    if word.isdigit():
                        lld[page][tb]['checks'].append(word)
                    elif '/' in word:
                        lld[page][tb]['dates'].append(word)
            if 'numbers2' in nn[page][tb].keys():
                txt = nn[page][tb]['numbers2']
                words = txt.split('\n')
                for word in words:
                    if word.isdigit():
                        lld[page][tb]['checks'].append(word)
            if 'rates' in nn[page][tb].keys():
                txt = nn[page][tb]['rates']
                words = txt.split('\n')
                for word in words:
                    if '.' in word:
                        lld[page][tb]['rates'].append(float(word))
            if 'positions' in nn[page][tb].keys():
                txt = nn[page][tb]['positions']
                words = txt.split('\n')
                for word in words:
                    if len(word)>1:
                        lld[page][tb]['positions'].append(word)
            if 'fund' in nn[page][tb].keys():
                lld[page][tb]['fund'] = nn[page][tb]['fund']
            if 'acct' in nn[page][tb].keys():
                lld[page][tb]['acct'] = nn[page][tb]['acct']
            if 'obj' in nn[page][tb].keys():
                lld[page][tb]['obj'] = nn[page][tb]['obj']
            if 'earnings' in nn[page][tb].keys():
                txt = nn[page][tb]['earnings']
                had_underscore = False
                words = txt.split('\n')
                for word in words:
                    if '.' in word:
                        if not had_underscore: 
                            lld[page][tb]['earnings'].append(float(word))
                            had_underscore = False
                    elif '_' in word:
                        had_underscore = True
            if (len(lld[page][tb]['checks']) < len(lld[page][tb]['dates'])):
                new_checks = []
                check_index = 0
                for i in np.arange(len(lld[page][tb]['earnings'])):
                    if (lld[page][tb]['earnings'][i] > 0.0):
                        new_checks.append(lld[page][tb]['checks'][check_index])
                        check_index += 1
                    else:
                        new_checks.append('gen'+str(page) + '-' + str(tb) + '-' + str(i))
                        print("inserting check number: ",page,tb,i)
                lld[page][tb]['checks'] = new_checks
    return(lld)

### Read the earnings reports and process them

In [23]:
def process_earnings(pdf):
    nnd = get_names(pdf)
    cnd = consolidate_name_boxes(nnd)
    newnames = combdd(cnd,pdf)
    lld = get_lines(newnames)
    return(lld)

ll={}

#ll[2015] = process_earnings(p15)
#ll[2016] = process_earnings(p16)
#ll[2017] = process_earnings(p17)
#ll[2018] = process_earnings(p18)
#ll[2019] = process_earnings(p19)
ll[2020] = process_earnings(p20)

inserting check number:  133 7 0
inserting check number:  133 7 3
inserting check number:  133 7 6
inserting check number:  136 11 0
inserting check number:  136 11 3
inserting check number:  140 7 0
inserting check number:  140 7 3
inserting check number:  143 9 0
inserting check number:  143 9 3
inserting check number:  143 9 5
inserting check number:  144 10 0
inserting check number:  148 7 0
inserting check number:  148 7 3
inserting check number:  148 7 5
inserting check number:  151 11 0
inserting check number:  152 10 0
inserting check number:  155 7 0
inserting check number:  155 7 3
inserting check number:  158 6 29
inserting check number:  159 11 5
inserting check number:  160 7 0
inserting check number:  177 7 4
inserting check number:  183 15 0
inserting check number:  249 7 0
inserting check number:  249 7 3
inserting check number:  257 12 2
inserting check number:  257 12 5
inserting check number:  257 12 8


### Check earnings against totals

In [24]:
totearn = {}

gtot = 0.0

for year in ll.keys():
    if year not in totearn.keys():
        totearn[year] = 0.0
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            for amt in ll[year][page][tb]['earnings']:
                totearn[year] += amt
    gtot += totearn[year]

#print(round(totearn[2015],2))       #FY2015 earnings report total is $20,527,796.79
#print(round(totearn[2016],2))       #FY2016 earnings report total is $21,988,876.13
#print(round(totearn[2017],2))       #FY2017 earnings report total is $22,608,024.34
#print(round(totearn[2018],2))       #FY2018 earnings report total is $22,409,915.41
#print(round(totearn[2019],2))       #FY2019 earnings report total is $23,372,079.04
print(round(totearn[2020],2))       #FY2020ytd earnings report total is $19,489,886.57
#print(gtot)

19489886.57


### Build a dictionary of people

In [69]:
people = {}

for year in ll.keys():
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            names         = ll[year][page][tb]['names']
            for i in np.arange(len(names)):
                name            = names[i]
                if (name not in people.keys()):
                    people[name] = pySC.Person(name)

people

{'STRAUT, MARYBETH': <SC_classes.Person at 0x7f682f22b9a0>,
 'RAKOVIC, EMILY E': <SC_classes.Person at 0x7f682ed8a940>,
 'FAY, ANN F': <SC_classes.Person at 0x7f682ed8a490>,
 'DOUCETTE, DANA C': <SC_classes.Person at 0x7f682ed8ab20>,
 'DEPASQUALE, HELEN L': <SC_classes.Person at 0x7f682ed8abe0>,
 'SIMONETTI, CATHERINE M': <SC_classes.Person at 0x7f682f0255e0>,
 'MATTOX, JENNIFER L': <SC_classes.Person at 0x7f682eed0670>,
 'HASKINS, RENEE L': <SC_classes.Person at 0x7f682eed0e50>,
 'DION, PETER R': <SC_classes.Person at 0x7f682eed09a0>,
 'PIERCE, CHERI-LYN': <SC_classes.Person at 0x7f682eed01f0>,
 'WOTHERSPOON, CAITLIN L': <SC_classes.Person at 0x7f682eed0040>,
 'OFFILER, LISA M': <SC_classes.Person at 0x7f682eed08e0>,
 'MCMULLEN, DEBRA S': <SC_classes.Person at 0x7f682eed0520>,
 'POTTER, NATHAN R': <SC_classes.Person at 0x7f682c06a1f0>,
 'SHERMAN, DANIELLE M': <SC_classes.Person at 0x7f682c06a610>,
 'MCGOVERN, LYNN K': <SC_classes.Person at 0x7f682c06a430>,
 'CENCI MEISER, MAURA': <SC_

### Build a list of check dates for each person

In [70]:
datelist = {}

for year in ll.keys():
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            names         = ll[year][page][tb]['names']
            check_dates   = ll[year][page][tb]['dates']
            
            for i in np.arange(len(names)):
                name            = names[i]
                if name not in datelist.keys():
                    datelist[name] = {}

                date_str        = check_dates[i]
                words = date_str.split('/')
                check_date   = date(int(words[2]),int(words[0]),int(words[1]))
                if (check_date not in datelist[name].keys()):
                    datelist[name][check_date] = 1
                    
for name in datelist.keys():
    print(name,datelist[name].keys())

STRAUT, MARYBETH dict_keys([datetime.date(2019, 7, 5), datetime.date(2019, 7, 19), datetime.date(2019, 8, 2), datetime.date(2019, 8, 16), datetime.date(2019, 8, 30), datetime.date(2019, 9, 13), datetime.date(2019, 9, 27), datetime.date(2019, 10, 11), datetime.date(2019, 10, 25), datetime.date(2019, 11, 8), datetime.date(2019, 11, 22), datetime.date(2019, 12, 6), datetime.date(2019, 12, 20), datetime.date(2020, 1, 3), datetime.date(2020, 1, 17), datetime.date(2020, 1, 31), datetime.date(2020, 2, 14), datetime.date(2020, 2, 28), datetime.date(2020, 3, 13), datetime.date(2020, 3, 27), datetime.date(2020, 4, 10)])
RAKOVIC, EMILY E dict_keys([datetime.date(2019, 7, 5), datetime.date(2019, 7, 19), datetime.date(2019, 8, 2), datetime.date(2019, 8, 16), datetime.date(2019, 8, 30), datetime.date(2019, 9, 13), datetime.date(2019, 9, 27), datetime.date(2019, 10, 11), datetime.date(2019, 10, 25), datetime.date(2019, 11, 8), datetime.date(2019, 11, 22), datetime.date(2019, 12, 6), datetime.date(201

FERREIRA, ANDREA J dict_keys([datetime.date(2019, 7, 5), datetime.date(2019, 7, 19), datetime.date(2019, 8, 2), datetime.date(2019, 8, 16), datetime.date(2019, 8, 30), datetime.date(2019, 9, 13), datetime.date(2019, 9, 27), datetime.date(2019, 10, 11), datetime.date(2019, 10, 25), datetime.date(2019, 11, 8), datetime.date(2019, 11, 22), datetime.date(2019, 12, 6), datetime.date(2019, 12, 20), datetime.date(2020, 1, 3), datetime.date(2020, 1, 17), datetime.date(2020, 1, 31), datetime.date(2020, 2, 14), datetime.date(2020, 2, 28), datetime.date(2020, 3, 13), datetime.date(2020, 3, 27), datetime.date(2020, 4, 10)])
LEMOI, TAMMY L dict_keys([datetime.date(2019, 7, 5), datetime.date(2019, 7, 19), datetime.date(2019, 8, 2), datetime.date(2019, 8, 16), datetime.date(2019, 8, 30), datetime.date(2019, 9, 13), datetime.date(2019, 9, 27), datetime.date(2019, 10, 11), datetime.date(2019, 10, 25), datetime.date(2019, 11, 8), datetime.date(2019, 11, 22), datetime.date(2019, 12, 6), datetime.date(201

### Add payperiod objects covering the check dates

In [71]:
datelist = {}

for year in ll.keys():
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            names         = ll[year][page][tb]['names']
            check_dates   = ll[year][page][tb]['dates']
            
            for i in np.arange(len(names)):
                name            = names[i]
                if name not in datelist.keys():
                    datelist[name] = {}

                date_str        = check_dates[i]
                words = date_str.split('/')
                check_date   = date(int(words[2]),int(words[0]),int(words[1]))
                if (check_date not in datelist[name].keys()):
                    datelist[name][check_date] = 1
                    
for name in datelist.keys():
    for ckdate in datelist[name].keys():
        pperiod = pySC.payperiod(ckdate)
        schyr = pperiod.get_school_year()
        syseq = pperiod.get_school_year_seq()
        print(name,schyr,syseq)
        people[name].add_payperiod_by_index(schyr,syseq,pperiod)

STRAUT, MARYBETH 2018-2019 28
STRAUT, MARYBETH 2018-2019 27
STRAUT, MARYBETH 2018-2019 26
STRAUT, MARYBETH 2019-2020 1
STRAUT, MARYBETH 2019-2020 2
STRAUT, MARYBETH 2019-2020 3
STRAUT, MARYBETH 2019-2020 4
STRAUT, MARYBETH 2019-2020 5
STRAUT, MARYBETH 2019-2020 6
STRAUT, MARYBETH 2019-2020 7
STRAUT, MARYBETH 2019-2020 8
STRAUT, MARYBETH 2019-2020 9
STRAUT, MARYBETH 2019-2020 10
STRAUT, MARYBETH 2019-2020 11
STRAUT, MARYBETH 2019-2020 12
STRAUT, MARYBETH 2019-2020 13
STRAUT, MARYBETH 2019-2020 14
STRAUT, MARYBETH 2019-2020 15
STRAUT, MARYBETH 2019-2020 16
STRAUT, MARYBETH 2019-2020 17
STRAUT, MARYBETH 2019-2020 18
RAKOVIC, EMILY E 2018-2019 28
RAKOVIC, EMILY E 2018-2019 27
RAKOVIC, EMILY E 2018-2019 26
RAKOVIC, EMILY E 2019-2020 1
RAKOVIC, EMILY E 2019-2020 2
RAKOVIC, EMILY E 2019-2020 3
RAKOVIC, EMILY E 2019-2020 4
RAKOVIC, EMILY E 2019-2020 5
RAKOVIC, EMILY E 2019-2020 6
RAKOVIC, EMILY E 2019-2020 7
RAKOVIC, EMILY E 2019-2020 8
RAKOVIC, EMILY E 2019-2020 9
RAKOVIC, EMILY E 2019-2020 1

SHERMAN, DANIELLE M 2019-2020 1
SHERMAN, DANIELLE M 2018-2019 28
SHERMAN, DANIELLE M 2018-2019 27
SHERMAN, DANIELLE M 2019-2020 2
SHERMAN, DANIELLE M 2019-2020 3
SHERMAN, DANIELLE M 2019-2020 4
SHERMAN, DANIELLE M 2019-2020 5
SHERMAN, DANIELLE M 2019-2020 6
SHERMAN, DANIELLE M 2019-2020 7
SHERMAN, DANIELLE M 2019-2020 8
SHERMAN, DANIELLE M 2019-2020 9
SHERMAN, DANIELLE M 2019-2020 10
SHERMAN, DANIELLE M 2019-2020 11
SHERMAN, DANIELLE M 2019-2020 12
SHERMAN, DANIELLE M 2019-2020 13
SHERMAN, DANIELLE M 2019-2020 14
SHERMAN, DANIELLE M 2019-2020 15
SHERMAN, DANIELLE M 2019-2020 16
SHERMAN, DANIELLE M 2019-2020 17
SHERMAN, DANIELLE M 2019-2020 18
MCGOVERN, LYNN K 2019-2020 11
CENCI MEISER, MAURA 2019-2020 11
CENCI MEISER, MAURA 2019-2020 14
CENCI MEISER, MAURA 2019-2020 16
CENCI MEISER, MAURA 2019-2020 18
CARTER, DONNA L 2019-2020 11
CARTER, DONNA L 2018-2019 28
CARTER, DONNA L 2018-2019 27
CARTER, DONNA L 2018-2019 26
CARTER, DONNA L 2019-2020 1
CARTER, DONNA L 2019-2020 2
CARTER, DONNA L

DELGALLO, ALLYSON C 2019-2020 14
DELGALLO, ALLYSON C 2019-2020 15
DELGALLO, ALLYSON C 2019-2020 16
DELGALLO, ALLYSON C 2019-2020 17
DELGALLO, ALLYSON C 2019-2020 18
GARCIA, ELLEN E 2018-2019 28
GARCIA, ELLEN E 2018-2019 27
GARCIA, ELLEN E 2018-2019 26
GARCIA, ELLEN E 2019-2020 1
GARCIA, ELLEN E 2019-2020 2
GARCIA, ELLEN E 2019-2020 3
GARCIA, ELLEN E 2019-2020 4
GARCIA, ELLEN E 2019-2020 5
GARCIA, ELLEN E 2019-2020 6
GARCIA, ELLEN E 2019-2020 7
GARCIA, ELLEN E 2019-2020 8
GARCIA, ELLEN E 2019-2020 9
GARCIA, ELLEN E 2019-2020 10
GARCIA, ELLEN E 2019-2020 11
GARCIA, ELLEN E 2019-2020 12
GARCIA, ELLEN E 2019-2020 13
GARCIA, ELLEN E 2019-2020 14
GARCIA, ELLEN E 2019-2020 15
GARCIA, ELLEN E 2019-2020 16
GARCIA, ELLEN E 2019-2020 17
GARCIA, ELLEN E 2019-2020 18
SULLIVAN, MARCY A 2018-2019 28
SULLIVAN, MARCY A 2018-2019 27
SULLIVAN, MARCY A 2018-2019 26
SULLIVAN, MARCY A 2019-2020 1
SULLIVAN, MARCY A 2019-2020 2
SULLIVAN, MARCY A 2019-2020 3
SULLIVAN, MARCY A 2019-2020 4
SULLIVAN, MARCY A 2019

STEFFY, JENNIFER A 2019-2020 1
STEFFY, JENNIFER A 2019-2020 2
STEFFY, JENNIFER A 2019-2020 3
STEFFY, JENNIFER A 2019-2020 4
STEFFY, JENNIFER A 2019-2020 5
STEFFY, JENNIFER A 2019-2020 6
STEFFY, JENNIFER A 2019-2020 7
STEFFY, JENNIFER A 2019-2020 8
STEFFY, JENNIFER A 2019-2020 9
STEFFY, JENNIFER A 2019-2020 10
STEFFY, JENNIFER A 2019-2020 11
STEFFY, JENNIFER A 2019-2020 12
STEFFY, JENNIFER A 2019-2020 13
STEFFY, JENNIFER A 2019-2020 14
STEFFY, JENNIFER A 2019-2020 15
STEFFY, JENNIFER A 2019-2020 16
STEFFY, JENNIFER A 2019-2020 17
STEFFY, JENNIFER A 2019-2020 18
KENNEY, TIMOTHY G 2018-2019 28
KENNEY, TIMOTHY G 2018-2019 27
KENNEY, TIMOTHY G 2018-2019 26
KENNEY, TIMOTHY G 2019-2020 1
KENNEY, TIMOTHY G 2019-2020 2
KENNEY, TIMOTHY G 2019-2020 3
KENNEY, TIMOTHY G 2019-2020 4
KENNEY, TIMOTHY G 2019-2020 5
KENNEY, TIMOTHY G 2019-2020 6
KENNEY, TIMOTHY G 2019-2020 7
KENNEY, TIMOTHY G 2019-2020 8
KENNEY, TIMOTHY G 2019-2020 9
KENNEY, TIMOTHY G 2019-2020 10
KENNEY, TIMOTHY G 2019-2020 11
KENNEY, 

ODAY, CHRISTINA M 2018-2019 26
ODAY, CHRISTINA M 2019-2020 1
ODAY, CHRISTINA M 2019-2020 2
ODAY, CHRISTINA M 2019-2020 3
ODAY, CHRISTINA M 2019-2020 4
ODAY, CHRISTINA M 2019-2020 5
ODAY, CHRISTINA M 2019-2020 6
ODAY, CHRISTINA M 2019-2020 7
ODAY, CHRISTINA M 2019-2020 8
ODAY, CHRISTINA M 2019-2020 9
ODAY, CHRISTINA M 2019-2020 10
ODAY, CHRISTINA M 2019-2020 11
ODAY, CHRISTINA M 2019-2020 12
ODAY, CHRISTINA M 2019-2020 13
ODAY, CHRISTINA M 2019-2020 14
ODAY, CHRISTINA M 2019-2020 15
ODAY, CHRISTINA M 2019-2020 16
ODAY, CHRISTINA M 2019-2020 17
ODAY, CHRISTINA M 2019-2020 18
REVKIN, BENJAMIN F 2018-2019 28
REVKIN, BENJAMIN F 2018-2019 27
REVKIN, BENJAMIN F 2018-2019 26
REVKIN, BENJAMIN F 2019-2020 1
REVKIN, BENJAMIN F 2019-2020 2
REVKIN, BENJAMIN F 2019-2020 3
REVKIN, BENJAMIN F 2019-2020 4
REVKIN, BENJAMIN F 2019-2020 5
REVKIN, BENJAMIN F 2019-2020 6
REVKIN, BENJAMIN F 2019-2020 7
REVKIN, BENJAMIN F 2019-2020 8
REVKIN, BENJAMIN F 2019-2020 9
REVKIN, BENJAMIN F 2019-2020 10
REVKIN, BENJA

...

BRAMAN, REBECCA J 2019-2020 17
BRAMAN, REBECCA J 2019-2020 18
AMERANTES, DENNIS A 2019-2020 3
AMERANTES, DENNIS A 2019-2020 4
AMERANTES, DENNIS A 2019-2020 5
AMERANTES, DENNIS A 2019-2020 6
AMERANTES, DENNIS A 2019-2020 7
AMERANTES, DENNIS A 2019-2020 8
AMERANTES, DENNIS A 2019-2020 9
AMERANTES, DENNIS A 2019-2020 10
AMERANTES, DENNIS A 2019-2020 12
AMERANTES, DENNIS A 2019-2020 13
AMERANTES, DENNIS A 2019-2020 14
AMERANTES, DENNIS A 2019-2020 15
AMERANTES, DENNIS A 2019-2020 16
AMERANTES, DENNIS A 2019-2020 17
HUGHES, LISA M 2018-2019 28
HUGHES, LISA M 2018-2019 27
HUGHES, LISA M 2018-2019 26
HUGHES, LISA M 2019-2020 1
HUGHES, LISA M 2019-2020 2
HUGHES, LISA M 2019-2020 3
HUGHES, LISA M 2019-2020 4
HUGHES, LISA M 2019-2020 5
HUGHES, LISA M 2019-2020 6
HUGHES, LISA M 2019-2020 7
HUGHES, LISA M 2019-2020 8
HUGHES, LISA M 2019-2020 9
HUGHES, LISA M 2019-2020 10
HUGHES, LISA M 2019-2020 11
HUGHES, LISA M 2019-2020 12
HUGHES, LISA M 2019-2020 13
HUGHES, LISA M 2019-2020 14
HUGHES, LISA M 2

COOKSON, GARY W 2019-2020 17
COOKSON, GARY W 2019-2020 18
CLEARY, EDWARD 2019-2020 13
CLEARY, EDWARD 2018-2019 28
CLEARY, EDWARD 2018-2019 27
CLEARY, EDWARD 2018-2019 26
CLEARY, EDWARD 2019-2020 1
CLEARY, EDWARD 2019-2020 2
CLEARY, EDWARD 2019-2020 3
CLEARY, EDWARD 2019-2020 4
CLEARY, EDWARD 2019-2020 5
CLEARY, EDWARD 2019-2020 6
CLEARY, EDWARD 2019-2020 7
CLEARY, EDWARD 2019-2020 8
CLEARY, EDWARD 2019-2020 9
CLEARY, EDWARD 2019-2020 10
CLEARY, EDWARD 2019-2020 11
CLEARY, EDWARD 2019-2020 12
CLEARY, EDWARD 2019-2020 14
CLEARY, EDWARD 2019-2020 15
CLEARY, EDWARD 2019-2020 16
CLEARY, EDWARD 2019-2020 17
CLEARY, EDWARD 2019-2020 18
BYRNE, JAMES M 2019-2020 12
BYRNE, JAMES M 2019-2020 3
BYRNE, JAMES M 2018-2019 28
BYRNE, JAMES M 2018-2019 27
BYRNE, JAMES M 2018-2019 26
BYRNE, JAMES M 2019-2020 1
BYRNE, JAMES M 2019-2020 2
BYRNE, JAMES M 2019-2020 4
BYRNE, JAMES M 2019-2020 5
BYRNE, JAMES M 2019-2020 6
BYRNE, JAMES M 2019-2020 7
BYRNE, JAMES M 2019-2020 8
BYRNE, JAMES M 2019-2020 9
BYRNE, J

LALLO, MICHAEL V 2019-2020 14
LALLO, MICHAEL V 2019-2020 15
LALLO, MICHAEL V 2019-2020 16
LALLO, MICHAEL V 2019-2020 17
LALLO, MICHAEL V 2019-2020 18
PARDEE, JAMES R 2018-2019 28
PARDEE, JAMES R 2018-2019 27
PARDEE, JAMES R 2018-2019 26
PARDEE, JAMES R 2019-2020 1
PARDEE, JAMES R 2019-2020 2
PARDEE, JAMES R 2019-2020 3
PARDEE, JAMES R 2019-2020 4
PARDEE, JAMES R 2019-2020 5
PARDEE, JAMES R 2019-2020 6
PARDEE, JAMES R 2019-2020 7
PARDEE, JAMES R 2019-2020 8
PARDEE, JAMES R 2019-2020 9
PARDEE, JAMES R 2019-2020 10
PARDEE, JAMES R 2019-2020 11
PARDEE, JAMES R 2019-2020 12
PARDEE, JAMES R 2019-2020 13
PARDEE, JAMES R 2019-2020 14
PARDEE, JAMES R 2019-2020 15
PARDEE, JAMES R 2019-2020 16
PARDEE, JAMES R 2019-2020 17
PARDEE, JAMES R 2019-2020 18
LASALLE, NICHOLAS 2018-2019 28
LASALLE, NICHOLAS 2019-2020 8
LASALLE, NICHOLAS 2018-2019 27
LASALLE, NICHOLAS 2018-2019 26
LASALLE, NICHOLAS 2019-2020 1
LASALLE, NICHOLAS 2019-2020 2
LASALLE, NICHOLAS 2019-2020 3
LASALLE, NICHOLAS 2019-2020 4
LASALLE

SUTHERLAND, KAREN L 2019-2020 5
SUTHERLAND, KAREN L 2019-2020 6
SUTHERLAND, KAREN L 2019-2020 7
SUTHERLAND, KAREN L 2019-2020 8
SUTHERLAND, KAREN L 2019-2020 9
SUTHERLAND, KAREN L 2019-2020 10
SUTHERLAND, KAREN L 2019-2020 11
SUTHERLAND, KAREN L 2019-2020 12
SUTHERLAND, KAREN L 2019-2020 13
SUTHERLAND, KAREN L 2019-2020 14
SUTHERLAND, KAREN L 2019-2020 15
SUTHERLAND, KAREN L 2019-2020 16
SUTHERLAND, KAREN L 2019-2020 17
SUTHERLAND, KAREN L 2019-2020 18
HEATH, JEFFREY 2018-2019 28
HEATH, JEFFREY 2018-2019 27
HEATH, JEFFREY 2018-2019 26
HEATH, JEFFREY 2019-2020 1
HEATH, JEFFREY 2019-2020 2
HEATH, JEFFREY 2019-2020 3
HEATH, JEFFREY 2019-2020 4
HEATH, JEFFREY 2019-2020 5
HEATH, JEFFREY 2019-2020 6
HEATH, JEFFREY 2019-2020 7
HEATH, JEFFREY 2019-2020 8
HEATH, JEFFREY 2019-2020 9
HEATH, JEFFREY 2019-2020 10
HEATH, JEFFREY 2019-2020 11
HEATH, JEFFREY 2019-2020 12
HEATH, JEFFREY 2019-2020 13
HEATH, JEFFREY 2019-2020 14
HEATH, JEFFREY 2019-2020 15
HEATH, JEFFREY 2019-2020 16
HEATH, JEFFREY 2019-

CLEARY, LYNN M 2019-2020 2
CLEARY, LYNN M 2019-2020 3
CLEARY, LYNN M 2019-2020 4
CLEARY, LYNN M 2019-2020 5
CLEARY, LYNN M 2019-2020 6
CLEARY, LYNN M 2019-2020 7
CLEARY, LYNN M 2019-2020 8
CLEARY, LYNN M 2019-2020 9
CLEARY, LYNN M 2019-2020 10
CLEARY, LYNN M 2019-2020 11
CLEARY, LYNN M 2019-2020 12
CLEARY, LYNN M 2019-2020 13
CLEARY, LYNN M 2019-2020 14
CLEARY, LYNN M 2019-2020 15
CLEARY, LYNN M 2019-2020 16
CLEARY, LYNN M 2019-2020 17
CLEARY, LYNN M 2019-2020 18
CAMBIO, SUSAN A 2018-2019 28
CAMBIO, SUSAN A 2018-2019 27
CAMBIO, SUSAN A 2018-2019 26
CAMBIO, SUSAN A 2019-2020 1
CAMBIO, SUSAN A 2019-2020 2
CAMBIO, SUSAN A 2019-2020 3
CAMBIO, SUSAN A 2019-2020 4
CAMBIO, SUSAN A 2019-2020 5
CAMBIO, SUSAN A 2019-2020 6
CAMBIO, SUSAN A 2019-2020 7
CAMBIO, SUSAN A 2019-2020 8
CAMBIO, SUSAN A 2019-2020 9
CAMBIO, SUSAN A 2019-2020 10
CAMBIO, SUSAN A 2019-2020 11
CAMBIO, SUSAN A 2019-2020 12
CAMBIO, SUSAN A 2019-2020 13
CAMBIO, SUSAN A 2019-2020 14
CAMBIO, SUSAN A 2019-2020 15
CAMBIO, SUSAN A 201

### Add checks to payperiod objects

In [72]:
for year in ll.keys():
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            check_numbers = ll[year][page][tb]['checks']
            names         = ll[year][page][tb]['names']
            check_dates   = ll[year][page][tb]['dates']
            
            for i in np.arange(len(check_numbers)):
                
                check_number    = check_numbers[i]
                name            = names[i]
                date_str        = check_dates[i]
                words = date_str.split('/')
                check_date   = date(int(words[2]),int(words[0]),int(words[1]))
                paycheck = pySC.pay_check(check_number,name,check_date)
                syear = paycheck.get_school_year()
                pprs = people[name].get_payperiods()
                for seq in pprs[syear].keys():
                    pday = pprs[syear][seq].get_payday()
                    ppday = pprs[syear][seq].get_previous_payday()
                    if ((ppday < check_date) & (check_date <= pday)):
                        pprs[syear][seq].add_check(check_number,paycheck)

### Add items to the checks

In [73]:
for year in ll.keys():
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            check_numbers = ll[year][page][tb]['checks']
            names         = ll[year][page][tb]['names']
            check_dates   = ll[year][page][tb]['dates']
            fund          = ll[year][page][tb]['fund']
            acct          = ll[year][page][tb]['acct']
            obj           = ll[year][page][tb]['obj']
            positions     = ll[year][page][tb]['positions']
            rates         = ll[year][page][tb]['rates']
            earnings      = ll[year][page][tb]['earnings']
            obj_desc      = UCOA_labels.get_label('Obj',obj)
            acct_desc     = EG_acct_codes.get_eg_acct_desc(acct)
            acct_UCOA     = EG_acct_codes.get_eg_acct_UCOA(acct)
            
            for i in np.arange(len(check_numbers)):
                
                check_number    = check_numbers[i]
                name            = names[i]
                date_str        = check_dates[i]
                position        = positions[i]
                rate            = rates[i]
                earned          = earnings[i]
                
                words = date_str.split('/')
                check_date   = date(int(words[2]),int(words[0]),int(words[1]))
                stepdata = {}
                if (position == 'TEACHER'):
                    stepdata = teacher_salary_matrix.decode_earnings(check_date, \
                        rate,earned,obj)
                    try:
                        l = len(stepdata['step'])
                    except KeyError:
                            stepdata={}
                pps = people[name].get_payperiods()
                for syear in pps.keys():
                    for seq in pps[syear].keys():
                        checks = pps[syear][seq].get_checks()
                        if (check_number in checks.keys()):
                            lineitem = pySC.check_lineitem(fund_,acct,obj, \
                                position,rate,earned,acct_desc,obj_desc, \
                                acct_UCOA,stepdata)
                            checks[check_number].add_item(lineitem)
                            

max exceeded:  421.2772 2019-12-06 7676.68
max exceeded:  421.2772 2020-01-17 7614.84
max exceeded:  469.7011 2020-03-13 5672.54
max exceeded:  406.6413 2020-01-31 4504.31
max exceeded:  463.337 2020-02-28 4134.39


In [74]:
itm_ct = 0

for name in people.keys():
    pps = people[name].get_payperiods()
    for syear in pps.keys():
        for seq in pps[syear].keys():
            checks = pps[syear][seq].get_checks()
            for ckno in checks.keys():
                itms = checks[ckno].get_items()
                for k in itms.keys():
                    itm_ct += 1
                    
itm_ct

12562